## Task 1

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
from skimage import io, transform
from PIL import Image

In [2]:
class imageDataset(Dataset):
    def __init__(self, df_train, root_dir, transform=None):
        self.landmarks_frame = df_train['imdbId'].astype(str) + '.jpg'
        self.root_dir = root_dir
        self.transform = transform
        self.df_train = df_train

    def __len__(self):
        return len(self.landmarks_frame)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = os.path.join(self.root_dir, self.landmarks_frame.iloc[idx])
        image = Image.open(img_name, 'r').convert('RGB')
        #genre = self.df_train.loc[idx,'genre']
        genre = np.array([0,0,0,0,0,0,0])
        for i in range(7):
            if(self.df_train.iloc[idx,i+1] == 1):
                genre[i] = 1
        genre = torch.from_numpy(genre)
        genre = genre.float()
        if self.transform:
            image = self.transform(image)
        #result = {'image': image, 'genre': genre}
        return image, genre

In [3]:
transform = transforms.Compose([#transforms.Resize(60),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5 ), (0.5, 0.5, 0.5))])
transform

Compose(
    ToTensor()
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
)

In [4]:
df_train = pd.read_csv('./pa2_data/part1_data/train.csv', sep=',')
df_train = df_train.drop_duplicates()
#df_train.set_index('imdbId',inplace=True)
#df_train = df_train[df_train==1].stack().reset_index().drop(0,1)
#df_train.columns = ['imdbId', 'genre']
train_set = imageDataset(df_train, root_dir = './pa2_data/part1_data/images/', transform=transform)

df_test = pd.read_csv('./pa2_data/part1_data/test.csv', sep=',')
test_set = imageDataset(df_test, root_dir = './pa2_data/part1_data/images/', transform=transform)

In [5]:
#print(len(dataset))
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
y = 0
'''for i in range(10):
    image, genre= train_set[i]
    temp = np.array(image)
    print(train_set.landmarks_frame[y])
    print(image.shape)
    print(genre)
    imshow((torchvision.utils.make_grid(image)))
    y = y + 1'''

'for i in range(10):\n    image, genre= train_set[i]\n    temp = np.array(image)\n    print(train_set.landmarks_frame[y])\n    print(image.shape)\n    print(genre)\n    imshow((torchvision.utils.make_grid(image)))\n    y = y + 1'

In [6]:
from torch.utils.data import random_split
train_size = int(0.8 * len(train_set))
valid_size = len(train_set) - train_size
full_train, validation = random_split(train_set, [train_size, valid_size])

train_loader = DataLoader(full_train, batch_size=1,
                          shuffle=True, num_workers=0)
valid_loader = DataLoader(validation, batch_size=1,
                          shuffle=True, num_workers=0)
test_loader = DataLoader(test_set, batch_size=1,
                         shuffle=True, num_workers=0)

In [7]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



'''class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=25)  
        # input channels = 3, 因為RGB所有3個channels, output為6個channels
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16,  kernel_size=17)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=40,  kernel_size=9)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(in_channels=40, out_channels=100,  kernel_size=5)
        self.fc1 = nn.Linear(100 * 9 * 3, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 7)
        #self.drops = nn.Dropout(0.3)
        #self.bn1 = nn.BatchNorm1d(d_in) #do batch normalizing transform
        #self.bn2 = nn.BatchNorm1d(128)
        
    def forward(self, x):
        #  3, 32, 32
        # out_dim = in_dim - kernel_size + 1  
        #32 - (5-1) -1 +1 =28
        x = self.pool(F.relu(self.conv1(x))) #6, 132, 89 
        x = self.pool(F.relu(self.conv2(x))) #16, 64, 43
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x))) 
        x = x.view(-1, 100 * 9 * 3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x'''
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)  
        # input channels = 3, 因為RGB所有3個channels, output為6個channels
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16,  kernel_size=5)  
        self.fc1 = nn.Linear(16 * 64 * 42, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 7)

    def forward(self, x):
        #  3, 32, 32
        # out_dim = in_dim - kernel_size + 1  
        #32 - (5-1) -1 +1 =28
        x = self.pool(F.relu(self.conv1(x))) #6, 14, 14 
        x = self.pool(F.relu(self.conv2(x))) #16, 5, 5
        x = x.view(-1, 16 * 64 * 42)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

In [8]:
model = Net()

In [9]:
from torchsummary import summary
summary(model, input_size=(3,268, 182))
#summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 6, 264, 178]             456
         MaxPool2d-2           [-1, 6, 132, 89]               0
            Conv2d-3          [-1, 16, 128, 85]           2,416
         MaxPool2d-4           [-1, 16, 64, 42]               0
            Linear-5                  [-1, 120]       5,161,080
            Linear-6                   [-1, 84]          10,164
            Linear-7                    [-1, 7]             595
Total params: 5,174,711
Trainable params: 5,174,711
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.56
Forward/backward pass size (MB): 4.35
Params size (MB): 19.74
Estimated Total Size (MB): 24.64
----------------------------------------------------------------


In [39]:
def save_checkpoint(save_path, model, optimizer, val_loss):
    if save_path==None:
        return
    save_path = save_path 
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'val_loss': val_loss}

    torch.save(state_dict, save_path)

    print(f'Model saved to ==> {save_path}')

def load_checkpoint(model, optimizer):
    save_path = f'cifar_net.pt'
    state_dict = torch.load(save_path)
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    val_loss = state_dict['val_loss']
    print(f'Model loaded from <== {save_path}')
    
    return val_loss



def TRAIN(net, train_loader, valid_loader,  num_epochs, eval_every, total_step, criterion, optimizer, val_loss, device, save_name):
    
    running_loss = 0.0
    running_corrects = 0
    running_num = 0
    global_step = 0
    if val_loss==None:
        best_val_loss = float("Inf")  
    else: 
        best_val_loss=val_loss
    

    for epoch in range(num_epochs):  # loop over the dataset multiple times

        for i, (inputs, labels) in enumerate(train_loader):
            net.train()
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            '''Training of the model'''
            # Forward pass
            outputs = net(inputs)
            #_, preds = torch.max(outputs.data, 1)
            for i in range(7):
                if(outputs[0][i] < 0.5):
                    outputs[0][i] = 0
                else:
                    outputs[0][i] = 1
            preds = outputs
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            global_step += 1

            running_loss += loss.item()
            running_corrects += torch.sum(preds == labels.data)
            #if(torch.sum(preds == labels.data)):
                #print('preds:', preds)
                #print('labels.data:', labels.data)
                #print('torch.sum(preds == labels.data):', torch.sum(preds == labels.data))
            running_num += 7
            #print(running_corrects)
            #print(running_num)

            '''Evaluating the model every x steps'''
            if global_step % eval_every == 0:
                with torch.no_grad():
                    net.eval()
                    val_running_loss = 0.0
                    val_running_corrects = 0
                    for val_inputs, val_labels in valid_loader: #do validation here
                        val_outputs = net(val_inputs)
                        val_loss = criterion(val_outputs, val_labels)
                        _, preds = torch.max(val_outputs.data, 1)
                        val_running_loss += val_loss.item()
                        val_running_corrects += torch.sum(preds == val_labels.data)

                    average_train_loss = running_loss / eval_every
                    average_val_loss = val_running_loss / len(valid_loader)
                    print('running_corrects: ', running_corrects)
                    print('running_num:',  float(running_num))
                    average_train_acc = running_corrects / float(running_num)
                    average_val_acc = val_running_corrects / float(len(valid_loader))

                    print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Train Acc: {:.4f}, Valid Loss: {:.4f},  Valid Acc: {:.4f}'
                          .format(epoch+1, num_epochs, global_step, total_step, average_train_loss,
                                  average_train_acc, average_val_loss, average_val_acc))

                    running_loss = 0.0
                    running_num = 0
                    running_corrects = 0
                    
                    if average_val_loss < best_val_loss:
                        best_val_loss = average_val_loss
                        save_checkpoint(save_name, net, optimizer, best_val_loss)#save the best model
    print('Finished Training')

In [40]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
num_epochs = 1
eval_every = 1000
total_step = len(train_loader)*num_epochs
best_val_loss = None
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
save_path = f'cifar_net.pt'
model = model.to(device)

TRAIN(model, train_loader, valid_loader, num_epochs, eval_every, total_step, criterion, optimizer, best_val_loss, device, save_path)

running_corrects:  tensor(5265)
running_num: 7000.0
Epoch [1/1], Step [1000/8132], Train Loss: 0.6931, Train Acc: 0.7521, Valid Loss: 0.6907,  Valid Acc: 1.7127
Model saved to ==> cifar_net.pt
running_corrects:  tensor(5270)
running_num: 7000.0
Epoch [1/1], Step [2000/8132], Train Loss: 0.6931, Train Acc: 0.7529, Valid Loss: 0.6907,  Valid Acc: 1.7127
running_corrects:  tensor(5271)
running_num: 7000.0
Epoch [1/1], Step [3000/8132], Train Loss: 0.6931, Train Acc: 0.7530, Valid Loss: 0.6907,  Valid Acc: 1.7127
running_corrects:  tensor(5264)
running_num: 7000.0
Epoch [1/1], Step [4000/8132], Train Loss: 0.6931, Train Acc: 0.7520, Valid Loss: 0.6907,  Valid Acc: 1.7127
running_corrects:  tensor(5282)
running_num: 7000.0
Epoch [1/1], Step [5000/8132], Train Loss: 0.6931, Train Acc: 0.7546, Valid Loss: 0.6907,  Valid Acc: 1.7127
running_corrects:  tensor(5262)
running_num: 7000.0
Epoch [1/1], Step [6000/8132], Train Loss: 0.6931, Train Acc: 0.7517, Valid Loss: 0.6907,  Valid Acc: 1.7127
ru

In [56]:
m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
print(m(input))
print(target)
output = loss(m(input), target)
output.backward()

a = np.zeros(5)

tensor([0.0950, 0.5425, 0.2409], grad_fn=<SigmoidBackward>)
tensor([1., 0., 0.])


array([0., 0., 0., 0., 0.])

In [14]:
print('asd'+'asd')

asdasd
